## DECISION TREES


1. **What is the approximate depth of a Decision Tree trained (without restrictions)
on a training set with 1 million instances?**

En los árboles de decisión la profundidad se calcula de forma logarítmica respecto al número de instancias, utilizando la fórmula log2(m). Por tanto, para un dataset de un millón de instancias, valdrá con un árbol de decisión de 20 nodos (aprox).

2. **Is a node’s Gini impurity generally lower or greater than its parent’s? Is it gener‐
ally lower/greater, or always lower/greater?**

La “Gini Impurity” de un nodo es generalmente menor que la del padre puesto que el algoritmo de entrenamiento (CHART) esta diseñado para dividir los nodos de manera que se minimice la suma ponderada de la impureza de sus hijos. Sin embargo, aunque el promedio ponderado de los hijos sea menor, puede ser que un hijo tenga una impureza mayor que el padre, siempre y cuando el otro hijo sea lo suficientemente “puro” para que el promedio total disminuya.

3. **If a Decision Tree is overfitting the training set, is it a good idea to try decreasing
max_depth?**

Si, es una buena decisión. Limitando la profundidad del árbol restringiremos su libertad, obligándolo a detenerse antes, creando reglas más general y simples que suelen funcionar mejor con datos nuevo que el modelo aun no ha visto.

4. **If a Decision Tree is underfitting the training set, is it a good idea to try scaling
the input features?**

No servirá de nada. Al contrario que las SVMs o los modelos lineales, los Árboles de Decisión son inmunes a la escala de las características, puesto que simplementen dividen los datos basándose en umbrales.

5. **If it takes one hour to train a Decision Tree on a training set containing 1 million
instances, roughly how much time will it take to train another Decision Tree on a
training set containing 10 million instances?**

El entrenamiento de un Árbol de decisión tiene una complejidad computacional de *O*(*n*×*m*log(*m*)), donde m es el número de instancias. Por tanto si entrenar un set de m = 1 millón cuesta una hora, entrenar uno de 10m costára aproximadamente 11.7 horas.

6. **If your training set contains 100,000 instances, will setting presort=True speed
up training?**

No, al contrario, relantizará el entrenamiento considerablemente y es e que pre-ordenar los datos solo es útil para acelerar el entrenamiento en conjuntos de datos pequeños (menos de unos poco miles de instancias), sino el algoritmo tiene qeu hacer un esfuero inmenso para ordenar todos los valores de las características antes de empezar a entrenar.

In [2]:
import numpy as np
from sklearn.datasets import make_moons, fetch_openml
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, ShuffleSplit
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.base import clone
from scipy.stats import mode
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.svm import LinearSVC, SVC

7. **Train and fine-tune a Decision Tree for the moons dataset.**
- **Generate a moons dataset using make_moons(n_samples=10000, noise=0.4).**

In [ ]:
# Importaciónd del dataset
X, y = make_moons(n_samples = 10000, noise=0.4, random_state=42)

- **Split it into a training set and a test set using train_test_split().**

In [4]:
# Separación en test de entrenamiento y test de prueba + entrenamiento
X_train, X_test, Y_train, Y_test = train_test_split(
    X, 
    y, 
    test_size=0.2,
    random_state=42)

- **Use grid search with cross-validation (with the help of the GridSearchCV class) to find good hyperparameter values for a DecisionTreeClassifier. Hint: try various values for max_leaf_nodes.**

In [5]:
#Validación cruzada y hiperparameter tuning
#Definición de los parámetros
param_grid = {
    "max_leaf_nodes": [5, 20, 50, 100, 200, 500], 
    "min_samples_split": [2, 5, 8, 12]
}

#Configuración del GridSearchSv
grid_search_cv = GridSearchCV(
    DecisionTreeClassifier(random_state=42), 
    param_grid, 
    verbose=1, 
    cv=3
)

- **Train it on the full training set using these hyperparameters, and measure your model’s performance on the test set. You should get roughly 85% to 87% accuracy.**

In [6]:
#Ejecución y obtención del mejor modelo
grid_search_cv.fit(X_train, Y_train)

best_model = grid_search_cv.best_estimator_
print(best_model)

#Medir la precisión final
y_pred = best_model.predict(X_test)

final_accuracy = accuracy_score(Y_test, y_pred)
print(f"Precisión final: {final_accuracy:.4f}")

Fitting 3 folds for each of 24 candidates, totalling 72 fits
DecisionTreeClassifier(max_leaf_nodes=20, random_state=42)
Precisión final: 0.8700


**8. Grow a forest.**

- Continuing the previous exercise, generate 1,000 subsets of the training set,
each containing 100 instances selected randomly. Hint: you can use ScikitLearn’s ShuffleSplit class for this.

In [ ]:
#Separación de set con instancias random
n_trees = 1000
n_instances = 100

subsets = []

rs = ShuffleSplit(n_splits=n_trees, test_size=len(X_train) - n_instances, random_state=42)

# El método split devuelve generadores de índices
for mini_train_index, mini_test_index in rs.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = Y_train[mini_train_index]
    subsets.append((X_mini_train, y_mini_train))

- Train one Decision Tree on each subset, using the best hyperparameter values found above. Evaluate these 1,000 Decision Trees on the test set. Since they were trained on smaller sets, these Decision Trees will likely perform worse than the first Decision Tree, achieving only about 80% accuracy.

In [ ]:
# Evaluar precisión en cada uno

forest = [clone(grid_search_cv.best_estimator_) for _ in range(n_trees)]

accuracy_scores = []

for tree, (X_mini_train, y_mini_train) in zip(forest, subsets):
    tree.fit(X_mini_train, y_mini_train)
    y_pred = tree.predict(X_test)
    accuracy = accuracy_score(Y_test, y_pred)
    accuracy_scores.append(accuracy)

print(f"Precisión media de los árboles individuales: {np.mean(accuracy_scores):.3f}")

- Now comes the magic. For each test set instance, generate the predictions of the 1,000 Decision Trees, and keep only the most frequent prediction (you can use SciPy’s mode() function for this). This gives you majority-vote predictions over the test set.

In [ ]:
#Calcular el voto mayoritario

Y_pred = np.empty([n_trees, len(X_test)], dtype=np.uint8)

for tree_index, tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(X_test)

y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)

- Evaluate these predictions on the test set: you should obtain a slightly higher accuracy than your first model (about 0.5 to 1.5% higher). Congratulations, you have trained a Random Forest classifier!

In [ ]:
#Calcular la probabilidad final
y_pred_majority_votes = y_pred_majority_votes.reshape([-1])

final_accuracy = accuracy_score(Y_test, y_pred_majority_votes)

print(f"Precisión Final: {final_accuracy:.4f}")